In [4]:
import pandas as pd
df=pd.read_csv("insurance_pre.csv")
df=pd.get_dummies(df,dtype=int,drop_first=True)
df

,age,bmi,children,charges,sex_male,smoker_yes
0,19,27.900,0,16884.92400,0,1
1,18,33.770,1,1725.55230,1,0
2,28,33.000,3,4449.46200,1,0
3,33,22.705,0,21984.47061,1,0
4,32,28.880,0,3866.85520,1,0
...,...,...,...,...,...,...
1333,50,30.970,3,10600.54830,1,0
1334,18,31.920,0,2205.98080,0,0
1335,18,36.850,0,1629.83350,0,0
1336,21,25.800,0,2007.94500,0,0


In [5]:
df.columns

Index(['age', 'bmi', 'children', 'charges', 'sex_male', 'smoker_yes'], dtype='object')

In [8]:
independent=df[['age', 'bmi', 'children', 'sex_male', 'smoker_yes']]
dependent=df[['charges']]
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(independent,dependent,test_size=0.20,random_state=0)

In [19]:
# Install LightGBM if not already installed
# pip install lightgbm
!pip install lightgbm

from lightgbm import LGBMRegressor
from sklearn.metrics import r2_score

# Base LightGBM model with good default hyperparameters
lgbm = LGBMRegressor(
    n_estimators=1000,
    learning_rate=0.05,
    max_depth=-1,
    num_leaves=31,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)

# Fit model
lgbm.fit(X_train, y_train)

# Predictions
y_pred = lgbm.predict(X_test)

# R² Score
print("Test R² Score:", r2_score(y_test, y_pred))


  Obtaining dependency information for lightgbm from https://files.pythonhosted.org/packages/5e/23/f8b28ca248bb629b9e08f877dd2965d1994e1674a03d67cd10c5246da248/lightgbm-4.6.0-py3-none-win_amd64.whl.metadata
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
    --------------------------------------- 0.0/1.5 MB 640.0 kB/s eta 0:00:03
   ---------- ----------------------------- 0.4/1.5 MB 5.9 MB/s eta 0:00:01
   ---------------------------------------- 1.5/1.5 MB 13.1 MB/s eta 0:00:00



[notice] A new release of pip is available: 23.2.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000199 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 313
[LightGBM] [Info] Number of data points in the train set: 1070, number of used features: 5
[LightGBM] [Info] Start training from score 13201.182046
Test R² Score: 0.8625186490584005


In [17]:
!pip install xgboost

from xgboost import XGBRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import r2_score
import numpy as np

# Flatten y to avoid warning
# Convert y_train and y_test to 1D NumPy arrays
y_train = y_train.values.ravel()
y_test = y_test.values.ravel()

# Base model
xgb = XGBRegressor(objective='reg:squarederror', random_state=42)

# Parameter grid for XGBoost
param_dist = {
    'n_estimators': [100, 200, 300, 500, 800, 1000],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'max_depth': [3, 5, 7, 10],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'gamma': [0, 0.1, 0.2, 0.3],
    'min_child_weight': [1, 3, 5]
}

# RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=xgb,
    param_distributions=param_dist,
    n_iter=50,
    cv=5,
    verbose=1,
    random_state=42,
    n_jobs=-1,
    scoring='r2'
)

# Fit
random_search.fit(X_train, y_train)

# Best parameters and score
print("Best Parameters:", random_search.best_params_)
print("Best CV R2 Score:", random_search.best_score_)

# Predict with best model
best_xgb = random_search.best_estimator_
y_pred = best_xgb.predict(X_test)

# Test R² score
print("Test R² Score:", r2_score(y_test, y_pred))


Fitting 5 folds for each of 50 candidates, totalling 250 fits



[notice] A new release of pip is available: 23.2.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Best Parameters: {'subsample': 0.8, 'n_estimators': 100, 'min_child_weight': 3, 'max_depth': 3, 'learning_rate': 0.05, 'gamma': 0.1, 'colsample_bytree': 0.8}
Best CV R2 Score: 0.8478183406563791
Test R² Score: 0.9012743456595077


In [21]:
import pickle
filename = "finalized_randomforest_xgb_regression.sav"
pickle.dump(best_xgb, open(filename, 'wb'))
loaded_model=pickle.load(open(filename,'rb'))

new_data = [[55, 32.780, 4, 1, 0]]
result = loaded_model.predict(new_data)

print("Prediction:", result[0])

Prediction: 15258.388


In [14]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import r2_score

# Pre-tuned Gradient Boosting Regressor
gbr = GradientBoostingRegressor(
    n_estimators=300,
    learning_rate=0.05,
    max_depth=4,
    min_samples_split=5,
    min_samples_leaf=2,
    subsample=0.9,
    random_state=42
)

# Fit model
gbr.fit(X_train, y_train)

# Predictions
y_pred_gbr = gbr.predict(X_test)

# R² score
r2_gbr = r2_score(y_test, y_pred_gbr)
print("Gradient Boosting R² Score:", r2_gbr)


C:\Users\thirg\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\ensemble\_gb.py:672: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)  # TODO: Is this still required?


Gradient Boosting R² Score: 0.8820883849335233


In [13]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

# Base model
rf = RandomForestRegressor(random_state=42)

# Parameter grid
param_dist = {
    'n_estimators': [100, 200, 300, 500],     # Number of trees
    'max_depth': [None, 5, 10, 15, 20],       # Tree depth
    'min_samples_split': [2, 5, 10],          # Min samples to split a node
    'min_samples_leaf': [1, 2, 4],            # Min samples at leaf
    'max_features': ['sqrt', 'log2', None],   # Features considered at split
    'bootstrap': [True, False]                # Bootstrapping method
}

# RandomizedSearchCV (faster than GridSearch)
random_search = RandomizedSearchCV(
    estimator=rf,
    param_distributions=param_dist,
    n_iter=50,          # Number of parameter combinations to try
    cv=5,               # Cross-validation folds
    verbose=2,
    random_state=42,
    n_jobs=-1,
    scoring='r2'
)

# Fit
random_search.fit(X_train, y_train)

# Best parameters and score
print("Best Parameters:", random_search.best_params_)
print("Best CV R² Score:", random_search.best_score_)

# Predict with best model
best_rf = random_search.best_estimator_
y_pred = best_rf.predict(X_test)

from sklearn.metrics import r2_score
print("Test R² Score:", r2_score(y_test, y_pred))


Fitting 5 folds for each of 50 candidates, totalling 250 fits


C:\Users\thirg\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:1365: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Best Parameters: {'n_estimators': 500, 'min_samples_split': 2, 'min_samples_leaf': 4, 'max_features': None, 'max_depth': 15, 'bootstrap': True}
Best CV R² Score: 0.835720785360151
Test R² Score: 0.8989203828052756


In [9]:
from sklearn.ensemble import RandomForestRegressor
regressor=RandomForestRegressor()
regressor.fit(X_train,y_train)

C:\Users\thirg\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:1365: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


,n_estimators,100
,criterion,'squared_error'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,1.0
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [11]:
y_pred=regressor.predict(X_test)

In [12]:
from sklearn.metrics import r2_score
r_score=r2_score(y_test,y_pred)
r_score

0.8797228425695149